In [1]:
import pandas as pd
from sqlalchemy import create_engine, VARCHAR
import os

# Connection parameters
server = 'PHONG'
database = 'Sales'
username = ''
password = ''

# Create a connection string
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=SQL+Server'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Directory containing CSV files
csv_directory = 'CSV_split2'

# Counter for tracking the number of files imported
files_imported = 0

# Iterate through CSV files in the directory
for csv_file in os.listdir(csv_directory):
    if csv_file.endswith('.csv'):
        csv_file_path = os.path.join(csv_directory, csv_file)

        # Extract table name from the CSV file name
        table_name = os.path.splitext(os.path.basename(csv_file_path))[0]

        # Read CSV file into a Pandas DataFrame
        df = pd.read_csv(csv_file_path, nrows=5)  # Read a few rows to infer data types

        # Create a new table based on the DataFrame structure
        df.to_sql(table_name, con=engine, if_exists='replace', index=False, dtype={col: VARCHAR(255) for col in df.columns})

        print(f"Table {table_name} created successfully.")

        # Read the entire CSV file into the table
        df = pd.read_csv(csv_file_path)
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)

        print(f"Data imported successfully into {table_name} table.")
        
        files_imported += 1

# Print the total number of files imported
print(f"{files_imported} files imported successfully.")


Table TT T01-2022_split_2 created successfully.
Data imported successfully into TT T01-2022_split_2 table.
Table TT T01-2023_split_2 created successfully.
Data imported successfully into TT T01-2023_split_2 table.
Table TT T02-2022_split_2 created successfully.


In [25]:
import pandas as pd
import os

def merge_csv_files_from_splits(split1_folder, split2_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of all CSV files in the first split folder
    split1_files = [f for f in os.listdir(split1_folder) if f.endswith('.csv')]

    # Loop through each CSV file in the first split folder
    for file1 in split1_files:
        file1_path = os.path.join(split1_folder, file1)
        file2 = file1.replace("_split_1", "_split_2")
        file2_path = os.path.join(split2_folder, file2)

        # Read CSV files into dataframes
        file1_df = pd.read_csv(file1_path)
        file2_df = pd.read_csv(file2_path)

        # Merge the dataframes
        merged_df = pd.concat([file1_df, file2_df], ignore_index=True)

        # Save the merged dataframe to a new CSV file with UTF-16 encoding
        output_name = f"Merged_{os.path.splitext(file1)[0]}.csv"
        output_path = os.path.join(output_folder, output_name)
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')

# Example usage with 2 split folders
split1_folder = "CSV_split1"
split2_folder = "CSV_split2"
output_folder = "Sales"

merge_csv_files_from_splits(split1_folder, split2_folder, output_folder)


In [ ]:
import pandas as pd
import os

def merge_csv_files_from_splits(split1_folder, split2_folder, productmaster_file, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of all CSV files in the first split folder
    split1_files = [f for f in os.listdir(split1_folder) if f.endswith('.csv')]

    # Read specific columns from the productmaster.csv file
    productmaster_df = pd.read_csv(productmaster_file, usecols=["product_id", "detail_product_group", "launch_season", "gender", "age_group", "activity_group"])

    # Loop through each CSV file in the first split folder
    for file1 in split1_files:
        file1_path = os.path.join(split1_folder, file1)
        file2 = file1.replace("_split_1", "_split_2")
        file2_path = os.path.join(split2_folder, file2)

        # Read CSV files into dataframes
        file1_df = pd.read_csv(file1_path)
        file2_df = pd.read_csv(file2_path)

        # Merge the dataframes based on 'product_id'
        merged_df = pd.concat([file1_df, file2_df], ignore_index=True)
        merged_df = pd.merge(merged_df, productmaster_df, on="product_id", how="left")

        # Save the merged dataframe to a new CSV file with UTF-8 encoding
        output_name = f"Merged_{os.path.splitext(file1)[0]}.csv"
        output_path = os.path.join(output_folder, output_name)
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')

# Example usage with 2 split folders and productmaster.csv file
split1_folder = "CSV_split1"
split2_folder = "CSV_split2"
productmaster_file = "Productmaster.csv"
output_folder = "Sales2"

merge_csv_files_from_splits(split1_folder, split2_folder, productmaster_file, output_folder)
